## Average JCT

## Import statements

In this notebook, we compare various heterogeneity-agnostic and heterogeneity-aware FIFO policies, with and without space sharing, on traces with single-GPU and multi-GPU jobs.

In [ ]:
from plotting_utils import plot_metric_vs_inverse_lambda, plot_jct_cdf
from utils import get_logfile_paths, average_jct_fn

## Get list of relevant logfiles and define label mapping

In [ ]:
logfile_paths = sorted(get_logfile_paths(
    "/future/u/deepakn/gavel/logs/single_gpu_continuous_jobs_final/"))

In [ ]:
labels = {"fifo": "FIFO",
          "fifo_perf": "Gavel",
          "fifo_packed": "Gavel w/ SS"}

## Plot average JCT vs. input job rate


In [ ]:
plot_metric_vs_inverse_lambda(
    logfile_paths,
    labels,
    v100s=36, p100s=36, k80s=36,
    policies=["fifo",
              "fifo_perf",
              "fifo_packed"],
    metric_fn=lambda x: average_jct_fn(x, min_job_id=4000, max_job_id=5000),
    metric_label="Average JCT\n(hours)",
    xmax=2.8,
    ymax=100.0,
    extrapolate=True,
    output_filename="continuous_jobs/multi_gpu_fifo_v100=36_p100=36_k80=36.pdf"
)

## Makespan

In this notebook, we compare various heterogeneity-agnostic and heterogeneity-aware makespan policies, with and without space sharing.

## Import statements

In [ ]:
from plotting_utils import *
from utils import get_logfile_paths, makespan_fn, prune

## Get list of relevant logfiles and define label mapping

In [ ]:
logfile_paths = sorted(
    get_logfile_paths(
        "/future/u/deepakn/gavel/logs/multi_gpu_static_jobs_final/", static_trace=True))

In [ ]:
labels = {"fifo": "FIFO",
          "gandiva": "Gandiva",
          "min_total_duration_perf": "Gavel",
          "min_total_duration_packed": "Gavel w/ SS"}

## Plotting functions

In [ ]:
def plot_metric_vs_num_total_jobs(logfile_paths,
                                  labels,
                                  v100s, p100s, k80s, policies, metric_fn,
                                  metric_label, xmax=None, ymax=None,
                                  output_filename=None):
    plt.figure(figsize=(8, 3))
    ax = plt.subplot2grid((1, 1), (0, 0), colspan=1)

    data = {"num_total_jobs": [], "metric": [], "seed": [],
            "policy": []}
    for policy in policies:
        relevant_logfile_paths = list(reversed(prune(
            logfile_paths, v100s, p100s, k80s, policy)))
        num_total_jobs = [x[0] for x in relevant_logfile_paths]
        metrics = [metric_fn(x[1]) for x in relevant_logfile_paths]
        seeds = [x[2] for x in relevant_logfile_paths]
        policies = [labels[policy] for i in range(len(metrics))]

        import pandas as pd
        data["num_total_jobs"] += num_total_jobs
        data["metric"] += metrics
        data["seed"] += seeds
        data["policy"] += policies
    import pandas as pd
    df = pd.DataFrame(data)
    print(df.groupby(["policy", "num_total_jobs"]).mean())

    sns.lineplot(x='num_total_jobs', y='metric', style='policy',
                 hue='policy',
                 data=data, ci='sd',
                 markers=True)

    ax.set_xlabel("Number of jobs")
    ax.set_ylabel(metric_label)
    ax.set_xlim([70, xmax])
    ax.set_xticks([100, 200, 300, 400, 500, 600, 700])
    ax.set_ylim([0, ymax])
    sns.despine()
    
    leg = plt.legend(frameon=False)
    bb = leg.get_bbox_to_anchor().inverse_transformed(ax.transAxes)
    bb.y0 += 0.22
    bb.y1 += 0.22
    leg.set_bbox_to_anchor(bb, transform=ax.transAxes)
    
    if output_filename is not None:
        with PdfPages(output_filename) as pdf:
            pdf.savefig(bbox_inches='tight')
    
    plt.show()

## Plot makespan vs. number of jobs

In [ ]:
print("V100s=36, P100s=36, K80s=36")
plot_metric_vs_num_total_jobs(
    logfile_paths,
    labels,
    v100s=36, p100s=36, k80s=36,
    policies=["fifo"],
    metric_fn=makespan_fn,
    metric_label="Makespan (hours)",
    xmax=None,
    ymax=800,
    output_filename="makespan/multi_gpu_v100=36_p100=36_k80=36.pdf"
)